In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ultralytics
#!pip install -U torch  ultralytics
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.4/612.4 kB 18.7 MB/s eta 0:00:00
--2023-06-27 07:57:00--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.227.219.33, 13.227.219.70, 13.227.219.59, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.227.219.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2564550879 (2.4G) [binary/octet-stream]
Saving to: ‘sam_vit_h_4b8939.pth’

sam_vit_h_4b8939.pt 100%[===================>]   2.39G   239MB/s    in 13s     

2023-06-27 07:57:13 (186 MB/s) - ‘sam_vit_h_4b8939.pth’ saved [2564550879/2564550879]



In [ ]:
from ultralytics import YOLO
import shutil
from ultralytics.yolo.data.annotator import auto_annotate
import cv2
import os
import numpy as np
from pathlib import Path
from ultralytics.vit.sam import PromptPredictor, build_sam
from ultralytics.yolo.utils.torch_utils import select_device
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt

# import & downloading the sam weights
import torch
import torchvision
print("PyTorch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)
print("CUDA is available:", torch.cuda.is_available())
import sys
!{sys.executable} -m pip install opencv-python matplotlib
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

from PIL import Image
sys.path.append("..")
from segment_anything import sam_model_registry, SamPredictor


PyTorch version: 2.0.1+cu118
Torchvision version: 0.15.2+cu118
CUDA is available: True
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-ualph1z_
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-ualph1z_
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
  Created wheel for segment-anything: filename=segment_anything-1.0-py3-none-any.whl size=36589 sha256=f4eed6fa6bed3d48c7758a53334c30ace53170a47585b00a76ff90092ca111c2
  Stored in directory: /tmp/pip-ephem-wheel-cache-zxx2w_hv/wheels/10/cf/59/9ccb2f0a1bcc81d4fbd0e501680b5d088d690c6cfbc02dc99d
Successfully built segm

In [ ]:
sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"
device = "cuda"
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)
predictor = SamPredictor(sam)
model=YOLO("/content/drive/MyDrive/olive/yolov8/runs/detect/train/weights/best.pt")


In [ ]:
def yolov8_detection(model, image_path):

    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = model(image, stream=True)  # generator of Results objects

    for result in results:
        boxes = result.boxes  # Boxes object for bbox outputs
        classes= boxes.cls.tolist()
    #print("BOXES",type(boxes))
    #print("classes",type(classes))
    #print("classes",len(classes))
    index=[]
    j=-1
    for box in result.boxes:
      j=j+1
      if (box.conf[0]<0.5):
        index.append(j)
        print(box.conf[0])
    bbox = boxes.xyxy.tolist()

    bbox = [[int(i) for i in box] for box in bbox]
    #print("BBOX",type(bbox))
    print("*************image_path",image_path)
    print("*************len(bbox)",len(bbox))
    print("*************index",index)
    c=-1
    for i in index:
      c=c+1
      i=i+c
      try :
        #print("befor",bbox)
        bbox.pop(i)
        classes.pop(i)
        #print("after",bbox)
      except :
        print("*****************************problem******************")
        continue

    return classes,bbox, image

#segmentation and bounding boxes

In [ ]:
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)

def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2))


In [ ]:
gc.collect()

242

In [ ]:
def yolov8_sam(predictor,image_path, output_path,model) :
  predictor=predictor
  image_path=image_path
  classes,yolov8_boxes, image = yolov8_detection(model, image_path)
  print("len(classes)",classes)
  print("len(yolov8_boxes)",len(yolov8_boxes))
  predictor.set_image(image)
  input_boxes = torch.tensor(yolov8_boxes, device=predictor.device)
  print("input boxes ",input_boxes)
  print(type(input_boxes))
  transformed_boxes = predictor.transform.apply_boxes_torch(input_boxes, image.shape[:2])
  masks, _, _ = predictor.predict_torch(
      point_coords=None,
      point_labels=None,
      boxes=transformed_boxes,
      multimask_output=False,
  )
  masks=masks.to(device="cpu")
  plt.figure(figsize=(10, 10))
  plt.imshow(image)
  i=-1
  for mask in masks:
      print(classes[i])
      i=i+1
      if(int(classes[i])==1):
        show_mask(mask.cpu().numpy(), plt.gca(), random_color=True)
  for box in input_boxes:
      show_box(box.cpu().numpy(), plt.gca())
  plt.axis('off')
  plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
  plt.close()



In [ ]:
yolov8_sam(predictor,"/content/drive/MyDrive/olive/data/im12.jpg", "/content/drive/MyDrive/olive/yolov8_SAM/image_bounding/im12.jpg",model)


0: 640x512 1 Tree, 60.2ms
Speed: 18.4ms preprocess, 60.2ms inference, 87.9ms postprocess per image at shape (1, 3, 640, 640)


len(classes) [1.0]
len(yolov8_boxes) 1
input boxes  tensor([[3074, 2683, 5906, 5759]], device='cuda:0')
<class 'torch.Tensor'>
1.0


In [ ]:
directory = "/content/drive/MyDrive/olive/data/"
output="/content/drive/MyDrive/olive/yolov8_SAM/image_bounding/"
for filename in os.listdir(directory):
  image_path=os.path.join(directory,filename)
  output_path=os.path.join(output,filename)
  yolov8_sam(predictor,image_path, output_path,model)


0: 640x512 1 Person, 2 Trees, 64.3ms
Speed: 15.9ms preprocess, 64.3ms inference, 34.8ms postprocess per image at shape (1, 3, 640, 640)


tensor(0.4484, device='cuda:0')
len(classes) [1.0, 0.0]
len(yolov8_boxes) 2
input boxes  tensor([[ 639,  581, 2929, 2356],
        [1181, 1786, 1538, 2569]], device='cuda:0')
<class 'torch.Tensor'>
0.0
1.0



0: 640x512 1 Person, 1 Tree, 9.4ms
Speed: 4.1ms preprocess, 9.4ms inference, 2.6ms postprocess per image at shape (1, 3, 640, 640)


len(classes) [1.0, 0.0]
len(yolov8_boxes) 2
input boxes  tensor([[ 499,  244, 2973, 2316],
        [1452, 1675, 1771, 2410]], device='cuda:0')
<class 'torch.Tensor'>
0.0
1.0



0: 640x512 1 Person, 1 Tree, 12.3ms
Speed: 4.2ms preprocess, 12.3ms inference, 3.4ms postprocess per image at shape (1, 3, 640, 640)


...

len(classes) [1.0, 0.0]
len(yolov8_boxes) 2
input boxes  tensor([[1697,  284, 3371, 1792],
        [1524, 1266, 1870, 2178]], device='cuda:0')
<class 'torch.Tensor'>
0.0
1.0



0: 480x640 2 Persons, 1 Tree, 17.7ms
Speed: 19.9ms preprocess, 17.7ms inference, 11.6ms postprocess per image at shape (1, 3, 640, 640)


tensor(0.2828, device='cuda:0')
len(classes) [1.0, 0.0]
len(yolov8_boxes) 2
input boxes  tensor([[1647,  433, 3177, 1815],
        [1454, 1287, 1847, 2353]], device='cuda:0')
<class 'torch.Tensor'>
0.0
1.0



0: 480x640 1 Person, 2 Trees, 200.3ms
Speed: 42.3ms preprocess, 200.3ms inference, 100.1ms postprocess per image at shape (1, 3, 640, 640)


len(classes) [1.0, 1.0, 0.0]
len(yolov8_boxes) 3
input boxes  tensor([[1416,  433, 2903, 1783],
        [   1,    0, 1096, 1024],
        [1510, 1216, 1853, 2074]], device='cuda:0')
<class 'torch.Tensor'>
0.0


#image segmentation with black bagroung

In [ ]:
def show_mask(mask, image, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]

    # Create a black background image
    black_image = np.zeros_like(image)

    # Apply the mask to the original image
    masked_image = np.where(mask.reshape(h, w, 1), image, black_image)

    # Display the masked image
    plt.imshow(masked_image)
    plt.axis('off')
    plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
    plt.close()

In [ ]:
def yolov8_sam(predictor,image_path, output_path,model) :
  predictor=predictor
  image_path=image_path
  classes,yolov8_boxes, image = yolov8_detection(model, image_path)
  print("len(classes)",classes)
  print("len(yolov8_boxes)",len(yolov8_boxes))
  predictor.set_image(image)
  input_boxes = torch.tensor(yolov8_boxes, device=predictor.device)
  print("input boxes ",input_boxes)
  print(type(input_boxes))
  transformed_boxes = predictor.transform.apply_boxes_torch(input_boxes, image.shape[:2])
  masks, _, _ = predictor.predict_torch(
      point_coords=None,
      point_labels=None,
      boxes=transformed_boxes,
      multimask_output=False,
  )
  masks=masks.to(device="cpu")


  i=-1
  for mask in masks:
      print(classes[i])
      i=i+1
      if(int(classes[i])==1):
        show_mask(mask.cpu().numpy(), image, random_color=True)
        break;





In [ ]:
directory = "/content/drive/MyDrive/olive/eq_augmented_data/part5/"
output="/content/drive/MyDrive/olive/yolov8_SAM/images_segment/"
for filename in os.listdir(directory):
  image_path=os.path.join(directory,filename)
  output_path=os.path.join(output,filename)
  yolov8_sam(predictor,image_path, output_path,model)


0: 640x480 3 Persons, 1 Tree, 57.5ms
Speed: 13.9ms preprocess, 57.5ms inference, 39.2ms postprocess per image at shape (1, 3, 640, 480)


tensor(0.3222, device='cuda:0')
*************image_path /content/drive/MyDrive/olive/eq_augmented_data/part5/IMG_20210603_074354.jpg
*************len(bbox) 4
*************index [3]
len(classes) [1.0, 0.0, 0.0]
len(yolov8_boxes) 3
input boxes  tensor([[ 395,  839, 2590, 2629],
        [2161, 2056, 2486, 3012],
        [2365, 2015, 2791, 3194]], device='cuda:0')
<class 'torch.Tensor'>
0.0



0: 640x480 1 Person, 1 Tree, 8.0ms
Speed: 3.3ms preprocess, 8.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 480)


*************image_path /content/drive/MyDrive/olive/eq_augmented_data/part5/im10_4.jpg
*************len(bbox) 2
*************index []
len(classes) [0.0, 1.0]
len(yolov8_boxes) 2
input boxes  tensor([[1353, 1220, 1782, 2102],
        [  10,  118, 1715, 1639]], device='cuda:0')
<class 'torch.Tensor'>
1.0
0.0



0: 640x480 1 Person, 1 Tree, 6.8ms
Speed: 5.6ms preprocess, 6.8ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 480)


...

*************image_path /content/drive/MyDrive/olive/eq_augmented_data/part5/im12_2_8_rotation_6.jpg
*************len(bbox) 2
*************index []
len(classes) [0.0, 1.0]
len(yolov8_boxes) 2
input boxes  tensor([[3302, 1496, 6178, 8084],
        [ 847, 3477, 3504, 5889]], device='cuda:0')
<class 'torch.Tensor'>
1.0
0.0



0: 640x512 1 Person, 1 Tree, 6.5ms
Speed: 7.9ms preprocess, 6.5ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 512)


*************image_path /content/drive/MyDrive/olive/eq_augmented_data/part5/im12_2_8_zoom_6.jpg
*************len(bbox) 2
*************index []
len(classes) [0.0, 1.0]
len(yolov8_boxes) 2
input boxes  tensor([[2770, 1230, 5157, 6783],
        [ 715, 2929, 2941, 4967]], device='cuda:0')
<class 'torch.Tensor'>
1.0
0.0



0: 640x512 1 Person, 1 Tree, 7.2ms
Speed: 6.3ms preprocess, 7.2ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 512)


*************image_path /content/drive/MyDrive/olive/eq_augmented_data/part5/im12_2_8_rotation_7.jpg
*************len(bbox) 2
*************index []
len(classes) [0.0, 1.0]
len(yolov8_boxes) 2
input boxes  tensor([[3283, 1547, 6199, 8078],
        [ 856, 3419, 3415, 5797]], device='cuda:0')
<class 'torch.Tensor'>
1.0
0.0


#Annotation  for the bounding box and the segmentation

In [ ]:
def yolov8_sam(predictor,image_path,model) :
  predictor=predictor
  image_path=image_path
  classes,yolov8_boxes, image = yolov8_detection(model, image_path)
  print("len(classes)",classes)
  print("len(yolov8_boxes)",len(yolov8_boxes))
  predictor.set_image(image)
  input_boxes = torch.tensor(yolov8_boxes, device=predictor.device)
  print("input boxes ",input_boxes)
  print(type(input_boxes))
  transformed_boxes = predictor.transform.apply_boxes_torch(input_boxes, image.shape[:2])
  masks, _, _ = predictor.predict_torch(
      point_coords=None,
      point_labels=None,
      boxes=transformed_boxes,
      multimask_output=False,
  )
  #print("len(masks)",len(masks))

  h1=0
  w1=0
  h2=0
  masks=masks.to(device="cpu")
  for i, mask in enumerate(masks):


      binary_mask = masks[i].squeeze().numpy().astype(np.uint8)

      # Find the contours of the mask
      contours, hierarchy = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

      largest_contour = max(contours, key=cv2.contourArea)

      # Get the new bounding box
      bbox = [int(x) for x in cv2.boundingRect(largest_contour)]

      # Get the segmentation mask for object
      segmentation = largest_contour.flatten().tolist()

      # Write bounding boxes to file in YOLO format
      for contour in contours:
        # Get the bounding box coordinates of the contour
        x, y, w, h = cv2.boundingRect(contour)
        if (classes[i]==1.0):
          if (h>h1):
            h1=h
            w1=w
        if (classes[i]==0.0):
          if (h>h2):
            h2=h



      mask=segmentation
      # load the image
      #width, height = image_path.size
      img = Image.open(image_path)
      width, height = img.size
      # convert mask to numpy array of shape (N,2)
      mask = np.array(mask).reshape(-1,2)

      # normalize the pixel coordinates
      mask_norm = mask / np.array([width, height])

      # compute the bounding boxlen(yolov8_boxes) 2
      xmin, ymin = mask_norm.min(axis=0)
      xmax, ymax = mask_norm.max(axis=0)
      bbox_norm = np.array([xmin, ymin, xmax, ymax])

      # concatenate bbox and mask to obtain YOLO format
      yolo = np.concatenate([bbox_norm, mask_norm.reshape(-1)])

      # compute the bounding box
      # write the yolo values to a text file
      if (int(classes[i])==1):
        with open('yolomask_format.txt', 'w') as f:
          f.write('0 ')
          for val in yolo:
            f.write("{:.6f} ".format(val))
          f.write('\n')
  try:
    Tree_heigth=(h1/h2)*1.7
    Tree_width=(w1/h1)*Tree_heigth
  except:
    print("************************problem*****************")
    Tree_heigth=0
    Tree_width=0
  return Tree_heigth,Tree_width



In [ ]:
directory = "/content/drive/MyDrive/olive/eq_augmented_data/part5/"
heigth=[]
width=[]
filename=[]
for filen in os.listdir(directory):
  print(filen)
  filename.append(filen)
  image_path=os.path.join(directory,filen)
  h,w =yolov8_sam(predictor,image_path,model)
  print("h=",h)
  print("w=",w)
  heigth.append(h)
  width.append(w)

IMG_20210603_074354.jpg



0: 640x480 3 Persons, 1 Tree, 11.1ms
Speed: 5.9ms preprocess, 11.1ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 480)


tensor(0.3222, device='cuda:0')
*************image_path /content/drive/MyDrive/olive/eq_augmented_data/part5/IMG_20210603_074354.jpg
*************len(bbox) 4
*************index [3]
len(classes) [1.0, 0.0, 0.0]
len(yolov8_boxes) 3
input boxes  tensor([[ 395,  839, 2590, 2629],
        [2161, 2056, 2486, 3012],
        [2365, 2015, 2791, 3194]], device='cuda:0')
<class 'torch.Tensor'>



0: 640x480 1 Person, 1 Tree, 12.7ms
Speed: 3.9ms preprocess, 12.7ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 480)


h= 2.693127147766323
w= 3.0976804123711337
im10_4.jpg
*************image_path /content/drive/MyDrive/olive/eq_augmented_data/part5/im10_4.jpg
*************len(bbox) 2
*************index []
len(classes) [0.0, 1.0]
len(yolov8_boxes) 2
input boxes  tensor([[1353, 1220, 1782, 2102],
        [  10,  118, 1715, 1639]], device='cuda:0')
<class 'torch.Tensor'>
h= 3.3143184421534935
w= 3.0962199312714773
im11_1.jpg


...

*************image_path /content/drive/MyDrive/olive/eq_augmented_data/part5/im12_2_8_rotation_6.jpg
*************len(bbox) 2
*************index []
len(classes) [0.0, 1.0]
len(yolov8_boxes) 2
input boxes  tensor([[3302, 1496, 6178, 8084],
        [ 847, 3477, 3504, 5889]], device='cuda:0')
<class 'torch.Tensor'>
h= 0.6284087411511234
w= 0.61088027085257
im12_2_8_zoom_6.jpg



0: 640x512 1 Person, 1 Tree, 10.9ms
Speed: 6.5ms preprocess, 10.9ms inference, 12.8ms postprocess per image at shape (1, 3, 640, 512)


*************image_path /content/drive/MyDrive/olive/eq_augmented_data/part5/im12_2_8_zoom_6.jpg
*************len(bbox) 2
*************index []
len(classes) [0.0, 1.0]
len(yolov8_boxes) 2
input boxes  tensor([[2770, 1230, 5157, 6783],
        [ 715, 2929, 2941, 4967]], device='cuda:0')
<class 'torch.Tensor'>
h= 0.5719684461566685
w= 0.6134470739313888
im12_2_8_rotation_7.jpg



0: 640x512 1 Person, 1 Tree, 7.9ms
Speed: 5.4ms preprocess, 7.9ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 512)


*************image_path /content/drive/MyDrive/olive/eq_augmented_data/part5/im12_2_8_rotation_7.jpg
*************len(bbox) 2
*************index []
len(classes) [0.0, 1.0]
len(yolov8_boxes) 2
input boxes  tensor([[3283, 1547, 6199, 8078],
        [ 856, 3419, 3415, 5797]], device='cuda:0')
<class 'torch.Tensor'>
h= 0.623106527093596
w= 0.6079279556650247


In [ ]:
import pandas as pd
# Create a dictionary from the lists
data = {'filename': filename, 'heigth': heigth, 'width': width}

# Create a DataFrame from the dictionary
df = pd.DataFrame(data)

# Save the DataFrame as a CSV file
df.to_csv('/content/drive/MyDrive/olive/eq_augmented_data/part5/yolov8_5.csv', index=False)

In [ ]:
import pandas as pd
import glob

# Get a list of all CSV files in the directory
csv_files = glob.glob('*.csv')

# Create an empty DataFrame to store the concatenated data
concatenated_data = pd.DataFrame()

# Iterate over each CSV file and append its data to the concatenated DataFrame
for file in csv_files:
    df = pd.read_csv(file)
    concatenated_data = concatenated_data.append(df, ignore_index=True)

# Save the concatenated data to a new CSV file
concatenated_data.to_csv('concatenated.csv', index=False)


<ipython-input-1-0c927be3b825>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  concatenated_data = concatenated_data.append(df, ignore_index=True)
<ipython-input-1-0c927be3b825>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  concatenated_data = concatenated_data.append(df, ignore_index=True)
<ipython-input-1-0c927be3b825>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  concatenated_data = concatenated_data.append(df, ignore_index=True)
<ipython-input-1-0c927be3b825>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  concatenated_data = concatenated_data.append(df, ignore_index=True)
<ipython-input-1-0c927be3b825>:13: FutureWarning